In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numba
import os

In [2]:
def b_rules(x):
    
    lasR, LasR, lasI, LasI, AI1, LasRAI1 = x
    
    k_lasR = 0.004
    g_lasR = 0.002
    k_LasR = 0.4
    g_LasR = 0.35
    a_lasI = 0.00036
    b_lasI = 0.0058
    K1 = 1.2
    h1 = -1.4
    g_lasI = 0.001
    k_LasI = 0.7
    g_LasI = 0.12
    k_AI1 = 1
    g_AI1 = 0.3
    u_LasRAI1 = 0.05
    s_LasRAI1 = 0.25
    g_LasRAI1 = 0.14
    # Cell parameters
    d = 0.8
    
    S0 = k_lasR                                       # + lasR
    S1 = lasR*g_lasR                                  # - lasR
    S2 = lasR*k_LasR                                  # + LasR
    S3 = LasRAI1*s_LasRAI1                            # + LasR ; + AI1 ; - LasRAI1
    S4 = AI1*LasR*u_LasRAI1                           # - LasR ; - AI1 ; + LasRAI1
    S5 = LasR*g_LasR                                  # - LasR
    S6 = a_lasI + (b_lasI/(1+(LasRAI1/K1)**h1))       # + lasI
    S7 = lasI*g_lasI                                  # - lasI
    S8 = lasI*k_LasI                                  # + LasI
    S9 = LasI*g_LasI                                  # - LasI
    S10 = LasI*k_AI1                                  # + AI1
    S11 = AI1*g_AI1                                   # - AI1
    S12 = d*AI1                                       # - AI1 ; + AI1_ext
    S13 = LasRAI1*g_LasRAI1                           # - LasRAI1
    
    Stot = S0 + S1 + S2 + S3 + S4 + S5 + S6 + S7 + S8 + S9 + S10 + S11 + S12 + S13
    
    U = np.random.rand()
    Tau = np.random.exponential(scale=1/Stot)
    
    if (U < S0/Stot):
        return np.array((U, Tau, 0), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U < (S0+S1)/Stot):
        return np.array((U, Tau, 1), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U < (S1+S2+S3)/Stot):
        return np.array((U, Tau, 2), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3)/Stot):
        return np.array((U, Tau, 3), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4)/Stot):
        return np.array((U, Tau, 4), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5)/Stot):
        return np.array((U, Tau, 5), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6)/Stot):
        return np.array((U, Tau, 6), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7)/Stot):
        return np.array((U, Tau, 7), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8)/Stot):
        return np.array((U, Tau, 8), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9)/Stot):
        return np.array((U, Tau, 9), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9+S10)/Stot):
        return np.array((U, Tau, 10), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9+S10+S11)/Stot):
        return np.array((U, Tau, 11), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9+S10+S11+S12)/Stot):
        return np.array((U, Tau, 12), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    else:
        return np.array((U, Tau, 13), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])

In [3]:
def e_rules(x):
    
    AI1_ext = x
    
    g_AI1_ext = 0.8
    d = 0.8
    d_away = 1.2
    
    if AI1_ext == 0:
        U = np.random.rand()
        Tau = np.random.exponential(scale=d+d_away+g_AI1_ext)                        # 100 is the temporal scale
        return np.array((U, Tau, 99), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])  # 99 denotes no reaction
    else:
        S14 = AI1_ext*d                       # + AI1 ; - AI1_ext
        S15 = AI1_ext*(d_away + g_AI1_ext)    # - AI1_ext
    
        Stot = S14 + S15
    
        U = np.random.rand()
        Tau = np.random.exponential(scale=1/Stot)
    
        if (U < S14/Stot):
            return np.array((U, Tau, 14), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
        else:
            return np.array((U, Tau, 15), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])

In [4]:
def multicompartmental_gillespie(N=10, steps=500, b_0=np.array([10, 10, 10, 10, 10, 10]), e_0=np.array([0])):
    
    if os.path.exists('bacteria.tsv'):
        os.remove('bacteria.tsv')
    if os.path.exists('environment.tsv'):
        os.remove('environment.tsv')
    
    # Initial conditions
    t = 0

    # Simulation initialization
    bacteria = np.zeros([N, len(b_0)])
    i = 0
    while i < N:
        x = 0
        while x < len(b_0):
            bacteria[i,x] = b_0[x]
            x += 1
        i += 1
    bacteria_file = pd.DataFrame(bacteria, columns=['lasR', 'LasR', 'lasI', 'LasI', 'AI1', 'LasRAI1'])
    bacteria_file.insert(0, 'cell', np.array(range(N)))
    bacteria_file.insert(0, 't', np.zeros(N))
    bacteria_file.set_index('t')
    bacteria_file.to_csv('bacteria.tsv', sep='\t', index=False)
    del bacteria_file

    env = np.zeros(len(e_0))
    x = 0
    while x < len(e_0):
        env[x] = e_0[x]
        x += 1
    env_file = pd.DataFrame(env, columns=['AI1_ext'])
    env_file.insert(0, 't', np.zeros(1))
    env_file.set_index('t')
    env_file.to_csv('environment.tsv', sep='\t', index=False)
    del env_file
    
    # Simulation
    step = 0
    while step < steps:
        reactions = np.zeros(N+1, dtype=[('tau','f4'), ('j','i4'), ('i','i4')])
        i = 0
        while i < N:
            reactions[i]['tau'] = b_rules(bacteria[i])['tau']
            reactions[i]['j'] = b_rules(bacteria[i])['j']
            reactions[i]['i'] = i
            i += 1
        reactions[N]['tau'] = e_rules(env[0])['tau']
        reactions[N]['j'] = e_rules(env[0])['j']
        reactions[N]['i'] = N
        
        reactions_sorted = np.sort(reactions, order='tau')
        
        h = 0
        t += reactions_sorted[0]['tau']
        for reaction_sorted in reactions_sorted:
            if reaction_sorted['i'] == N:
                if reaction_sorted['j'] == 99:
                    t += reaction_sorted['tau']
                    env[0] = env[0]
                    with open('environment.tsv', 'a') as env_file:
                        env_file.write(f'{t}\t')
                        env_file.write(f'{env[0]}\t')
                        env_file.write(f'\n')
                elif reaction_sorted['j'] == 14:
                    t += reaction_sorted['tau']
                    i = np.random.randint(0,N)
                    bacteria[i,4] = bacteria[i,4]+1
                    env[0] = env[0]-1
                    with open('environment.tsv', 'a') as env_file:
                        env_file.write(f'{t}\t')
                        env_file.write(f'{env[0]}\t')
                        env_file.write(f'\n')
                else:
                    t += reaction_sorted['tau']
                    env[0] = env[0]-1
                    with open('environment.tsv', 'a') as env_file:
                        env_file.write(f'{t}\t')
                        env_file.write(f'{env[0]}\t')
                        env_file.write(f'\n')

            else:
                if reaction_sorted['j'] == 0:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,0] = bacteria[i,0]+1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 1:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,0] = bacteria[i,0]-1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 2:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,1] = bacteria[i,1]+1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 3:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,1] = bacteria[i,1]+1
                    bacteria[i,4] = bacteria[i,4]+1
                    bacteria[i,5] = bacteria[i,5]-1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 4:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,1] = bacteria[i,1]-1
                    bacteria[i,4] = bacteria[i,4]-1
                    bacteria[i,5] = bacteria[i,5]+1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 5:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,1] = bacteria[i,1]-1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 6:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,2] = bacteria[i,2]+1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 7:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,2] = bacteria[i,2]-1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 8:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,3] = bacteria[i,3]-1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 9:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,3] = bacteria[i,3]-1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 10:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,4] = bacteria[i,4]+1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 11:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,4] = bacteria[i,4]-1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                elif reaction_sorted['j'] == 12:
                    t += reaction_sorted['tau']
                    env[0] = env[0]+1
                    i = reaction_sorted['i']
                    bacteria[i,4] = bacteria[i,4]+1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
                    with open('environment.tsv', 'a') as env_file:
                        env_file.write(f'{t}\t')
                        env_file.write(f'{env[0]}\t')
                        env_file.write(f'\n')
                else:
                    t += reaction_sorted['tau']
                    i = reaction_sorted['i']
                    bacteria[i,5] = bacteria[i,5]-1
                    with open('bacteria.tsv', 'a') as bacteria_file:
                        bacteria_file.write(f'{t}\t')
                        bacteria_file.write(f'{i}\t')
                        for x in bacteria[i]:
                            bacteria_file.write(f'{x}\t')
                        bacteria_file.write(f'\n')
            h += 1
        step += 1
    print('Finish')

In [5]:
multicompartmental_gillespie()

Finish
